In [0]:
# pip install loguru

  Obtaining dependency information for loguru from https://files.pythonhosted.org/packages/0c/29/0348de65b8cc732daa3e33e67806420b2ae89bdce2b04af740289c5c6c8c/loguru-0.7.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/61.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 61.4/61.6 kB 3.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 909.8 kB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import sys

repo_path = "/Workspace/Users/opolo.holtz@amaris.com/.bundle/marvelous-databricks-course-OpoloHOLTZ/dev/files/src"
sys.path.append(repo_path)

# Vérifier que le chemin est bien pris en compte
print(sys.path)

['/databricks/python_shell/scripts', '/local_disk0/spark-5459f246-a352-4f35-bcac-facdaef40796/userFiles-875db54d-0af0-461c-9480-1e7037dd728e', '/databricks/spark/python', '/databricks/spark/python/lib/py4j-0.10.9.7-src.zip', '/databricks/jars/spark--driver--driver-spark_3.5_2.12_deploy.jar', '/databricks/python_shell/lib', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '/local_disk0/.ephemeral_nfs/envs/pythonEnv-93a24c64-c1ab-477b-9037-059fbf947c5f/lib/python3.11/site-packages', '/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages', '/databricks/python/lib/python3.11/site-packages', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/databricks/.python_edge_libs', '', '/Workspace/Users/opolo.holtz@amaris.com/.bundle/marvelous-databricks-course-OpoloHOLTZ/dev/files/notebooks', '/Workspace/Users/opolo.holtz@amaris.com/.bundle/marvelous-databricks-course-OpoloHOLTZ/dev/files/src']


In [0]:
import os
import time
from typing import List, Dict

import requests
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from defaultccc.config import ProjectConfig
from defaultccc.serving.model_serving import ModelServing

# spark session
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

# Load project config
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
catalog_name = config.catalog_name
schema_name = config.schema_name

In [0]:
# Initialize feature store manager
model_serving = ModelServing(
    model_name=f"{catalog_name}.{schema_name}.default_ccc_model_basic", endpoint_name="default_ccc-model-serving"
)

In [0]:
# Deploy the model serving endpoint
model_serving.deploy_or_update_serving_endpoint()

2025-03-06 15:22:40.870 | INFO     | defaultccc.serving.model_serving:get_latest_model_version:24 - Latest model version: 1


In [0]:
# Create a sample request body
required_columns = config.num_features + config.cat_features

# Sample 1000 records from the training set
test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").toPandas()

# Sample 100 records from the training set
sampled_records = test_set[required_columns].sample(n=100, replace=True).to_dict(orient="records")
dataframe_records = [[record] for record in sampled_records]


In [0]:
dataframe_records[0]

[{'LIMIT_BAL': 380000,
  'BILL_AMT1': 0,
  'BILL_AMT2': 292,
  'BILL_AMT3': 7352,
  'BILL_AMT4': 3401,
  'BILL_AMT5': 1042,
  'BILL_AMT6': 4000,
  'PAY_AMT1': 292,
  'PAY_AMT2': 7352,
  'PAY_AMT3': 3401,
  'PAY_AMT4': 1042,
  'PAY_AMT5': 4000,
  'PAY_AMT6': 5000,
  'AGE': '20-30',
  'SEX': 2,
  'EDUCATION': 1,
  'MARRIAGE': 2,
  'PAY_0': -2,
  'PAY_2': -2,
  'PAY_3': -2,
  'PAY_4': -2,
  'PAY_5': -2,
  'PAY_6': -2}]

In [0]:
model_serving.call_endpoint(dataframe_records[0])

(200, {'predictions': [0]})

In [0]:
model_serving.evaluate_serving_model(test_set[required_columns + [config.target]].sample(n=100, replace=True), config.target)

2025-03-07 09:35:31.315 | INFO     | defaultccc.serving.model_serving:evaluate_serving_model:86 - Evaluating the deployed model via the serving endpoint...
2025-03-07 09:35:31.832 | INFO     | defaultccc.serving.model_serving:evaluate_serving_model:103 - Serving Model Evaluation: {'accuracy': 0.78, 'precision': 0.391304347826087, 'recall': 0.5294117647058824, 'f1_score': 0.45, 'roc_auc': 0.6803685329553507, 'confusion_matrix': [[69, 14], [8, 9]]}


{'accuracy': 0.78,
 'precision': 0.391304347826087,
 'recall': 0.5294117647058824,
 'f1_score': 0.45,
 'roc_auc': 0.6803685329553507,
 'confusion_matrix': [[69, 14], [8, 9]]}